In [1]:
import os, sys
import numpy as np 
import pandas as pd
from pymongo import MongoClient


In [2]:
client = MongoClient('mongodb://146.203.54.131:27017/L1000FWD')
coll_sigs = client['L1000FWD']['sigs']


In [4]:
cur = coll_sigs.find({}, {'_id':False, 'sig_id':True, 'pert_dose':True, 'pert_time':True, 'batch':True})
print cur.count()

sig_meta_df = pd.DataFrame([doc for doc in cur]).set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

127963
(127963, 3)


,batch,pert_dose,pert_time
sig_id,,,
LJP001_HS578T_24H:BRD-K59369769-001-11-2:0.4,LJP001_HS578T_24H,0.40,24.0
LJP001_HS578T_24H:BRD-K84085265-001-02-1:0.4,LJP001_HS578T_24H,0.40,24.0
LJP001_HS578T_24H:BRD-K36740062-001-02-5:2,LJP001_HS578T_24H,2.00,24.0
LJP001_HS578T_24H:BRD-A68739437-001-02-8:0.08,LJP001_HS578T_24H,0.08,24.0
LJP001_HS578T_24H:BRD-K52911425-001-04-9:2,LJP001_HS578T_24H,2.00,24.0


In [5]:
sig_meta_df.count()

batch        127963
pert_dose    127963
pert_time    127518
dtype: int64

In [6]:
sig_meta_df.loc[sig_meta_df['pert_time'].isnull()]['batch'].unique()

array([u'NMH001_NEU.KCL_6H.4H', u'NMH001_NEU.KCL_24H.4H',
       u'NMH002_NEU.KCL_24H.4H', u'NMH002_NEU.KCL_6H.4H'], dtype=object)

In [9]:
# Fill pert_time for sig_ids with missing pert_time
sig_meta_df_missing_time = sig_meta_df.loc[sig_meta_df['pert_time'].isnull()]

for sig_id, row in sig_meta_df_missing_time.iterrows():
    pert_time = row['batch'].split('_')[-1].split('.')[0].strip('H')
    pert_time = int(pert_time)
    coll_sigs.update_one({'sig_id': sig_id}, {'$set': {'pert_time': pert_time}})    
#     print sig_id, pert_time


In [10]:
cur = coll_sigs.find({}, {'_id':False, 'sig_id':True, 
                          'mean_cosine_dist_centered_by_batch':True, 
                          'SCS_centered_by_batch':True})
print cur.count()

sig_meta_df = pd.DataFrame([doc for doc in cur]).set_index('sig_id')
print sig_meta_df.shape
sig_meta_df.head()

127963
(127963, 2)


,SCS_centered_by_batch,mean_cosine_dist_centered_by_batch
sig_id,,
LJP001_HS578T_24H:BRD-K59369769-001-11-2:0.4,0.4384,0.996637
LJP001_HS578T_24H:BRD-K84085265-001-02-1:0.4,0.4384,0.996685
LJP001_HS578T_24H:BRD-K36740062-001-02-5:2,0.0637,0.958513
LJP001_HS578T_24H:BRD-A68739437-001-02-8:0.08,0.8908,1.032636
LJP001_HS578T_24H:BRD-K52911425-001-04-9:2,0.0028,0.909673


In [11]:
sig_meta_df.count()

SCS_centered_by_batch                 127963
mean_cosine_dist_centered_by_batch    127176
dtype: int64

In [12]:
sig_meta_df.loc[sig_meta_df['mean_cosine_dist_centered_by_batch'].isnull()].head()

,SCS_centered_by_batch,mean_cosine_dist_centered_by_batch
sig_id,,
CPC012_A375_6H:BRD-K88710727-001-01-4:10,0.0,NaN
PRISM001_A375_6H:BRD-K36740062-001-02-5:0.625,0.0,NaN
PRISM001_A375_6H:BRD-K59369769-001-05-4:5,0.0,NaN
PRISM001_A375_6H:BRD-K83963101-001-01-0:5,0.0,NaN
PRISM001_A375_6H:BRD-K83963101-001-01-0:0.312,0.0,NaN


In [14]:
doc = next(coll_sigs.find())
print doc.keys()

[u'CDavg_center_LM_det', u'cell_id', u'mean_cosine_dist_centered_by_batch', u'SCS_centered_by_batch', u'distil_id', u'avg_center_LM', u'batch', u'pert_dose', u'pert_desc', u'pert_id', u'CD_center_LM', u'avg_center_LM_det', u'sig_id', u'pert_time', u'CD_nocenter_LM', u'_id', u'CD_center_LM_det', u'CDavg_nocenter_LM_det']
